### Project 1

#### Part A

Jerônimo de Abreu Afrange

- Select 30 stocks in the S&P 500 universe
- Collect daily returns for the past 10 years (2014-2023)
- Create both value-weighted and equally weighted portfolio
- Rebalance these portfolios in the first day of the month based on the information on the last day (1-day lag)
- Which portfolio has the highest turnover?
- Compute daily returns for both portfolios
- Compute the following statistics for these portfolios:
    - Annualized average return
    - Annualized standar deviation
    - Sharpe ratio
    - Information ratio (vs S&P500)
- Plot cumulative returns for both portfolios, S&P500 and risk-free




In [1]:
import pandas as pd
import dateutil.relativedelta

In [2]:
# AUXILIARY FUNCTIONS

# calculates the month number
def month_number(date, start_date):
    delta = dateutil.relativedelta.relativedelta(date, start_date)
    return delta.years * 12 + delta.months

In [3]:
# DATA LOADING AND INITIAL MANIPULATION

# creates the DataFrames for de S&P500 index and de stock prices
stocks_data = pd.read_csv("../data/stock_prices.csv")
index_data = pd.read_csv("../data/sp500_index.csv")

# removes the stocks of which we don't have data from 2014 to the current day
incomplete_stocks = stocks_data["Ticker"].value_counts()
incomplete_stocks = incomplete_stocks[incomplete_stocks < incomplete_stocks.max()]
stocks_data = stocks_data[~stocks_data["Ticker"].isin(incomplete_stocks.index)]

In [4]:
# CALCULATION OF THE DAILY RETURNS AND CREATION OF THE MONTH NUMBER COLUMN

# creation of the daily return column
stocks_data["RETURN"] = stocks_data["PX_LAST"] / stocks_data["PX_OPEN"] - 1

# creation of the month number column
stocks_data["Date"] = pd.to_datetime(stocks_data["Date"])
stocks_data["MONTH_NUMBER"] = stocks_data["Date"].apply(lambda x: month_number(x, stocks_data["Date"].min()))

# defines a few constants
INITIAL_DATE = stocks_data["Date"].min()
INITIAL_PORT_VALUE = 1e6

In [7]:
# CREATION OF THE VALUE WEIGHTED PORTFOLIO

# creates the porfolio DataFrame, which is a copy of the current stocks_data DataFrame
portfolio_vw = stocks_data.copy()

# convenience variables
initial_date_filter = portfolio_vw["Date"] == INITIAL_DATE

# creation of the column for the total market cap of the stocks and column of the share of the total
portfolio_vw["TOTAL_MKT_CAP"] = portfolio_vw.groupby("Date")["CUR_MKT_CAP"].transform("sum")
portfolio_vw["MKT_CAP_SHARE"] = portfolio_vw["CUR_MKT_CAP"] / portfolio_vw["TOTAL_MKT_CAP"]

# creation of the initial porfolio stock ammount for each stock, value for each stock position and total portfolio value
portfolio_vw.loc[initial_date_filter, "PORT_STOCK_AMT"] = (INITIAL_PORT_VALUE * portfolio_vw["MKT_CAP_SHARE"]) / portfolio_vw["PX_OPEN"]
portfolio_vw.loc[initial_date_filter, "PORT_STOCK_VAL_LAST"] = portfolio_vw["PX_LAST"] * portfolio_vw["PORT_STOCK_AMT"]
portfolio_vw.loc[initial_date_filter, "PORT_VAL_LAST"] = portfolio_vw.groupby("Date")["PORT_STOCK_VAL_LAST"].transform("sum")


In [8]:
portfolio_vw[portfolio_vw["Date"] == INITIAL_DATE]

,Date,PX_LAST,PX_OPEN,PX_HIGH,PX_LOW,PX_VOLUME,CUR_MKT_CAP,Ticker,RETURN,MONTH_NUMBER,TOTAL_MKT_CAP,MKT_CAP_SHARE,PORT_STOCK_AMT,PORT_STOCK_VAL_LAST,PORT_VAL_LAST
0,2014-01-02,76.2700,76.0400,76.5000,75.9000,5212460.0,134027.7199,DIS,0.003025,0,1.005353e+06,0.133314,1753.209324,133314.037014,1000000.0
2670,2014-01-02,80.6200,80.8400,81.8900,80.2000,544022.0,7918.3920,UHS,-0.002721,0,1.005353e+06,0.007876,97.429830,7876.227432,1000000.0
5340,2014-01-02,27.8550,27.9140,27.9710,27.7330,36454380.0,371879.3863,GOOGL,-0.002114,0,1.005353e+06,0.369899,13251.385306,369899.169418,1000000.0
8010,2014-01-02,49.4400,49.5500,49.8600,49.1500,5458567.0,51705.7858,LOW,-0.002220,0,1.005353e+06,0.051430,1037.950720,51430.458170,1000000.0
10680,2014-01-02,11.9100,12.0200,12.0200,11.8500,7971602.0,15902.4873,BSX,-0.009151,0,1.005353e+06,0.015818,1315.957429,15817.808302,1000000.0
16212,2014-01-02,82.5100,83.2850,83.9781,82.0300,2944140.0,45046.2851,EOG,-0.009305,0,1.005353e+06,0.044806,537.989057,44806.418580,1000000.0
18882,2014-01-02,88.4200,89.2000,89.6900,87.6600,1314064.0,11352.0576,SBAC,-0.008744,0,1.005353e+06,0.011292,126.587546,11291.609140,1000000.0
21552,2014-01-02,73.1600,73.3500,73.5550,72.9200,1733439.0,32117.3293,STT,-0.002590,0,1.005353e+06,0.031946,435.532489,31946.308049,1000000.0
24222,2014-01-02,61.1600,61.6900,61.8400,60.9200,2061600.0,24830.9600,DVN,-0.008591,0,1.005353e+06,0.024699,400.368583,24698.737866,1000000.0
26892,2014-01-02,73.8300,74.0700,74.1500,72.8800,966843.0,17258.2695,VRTX,-0.003240,0,1.005353e+06,0.017166,231.758757,17166.371111,1000000.0
